In [1]:
from lark import InlineTransformer
from lark import Lark

In [77]:
grammar = r"""

    ?start : pipe
    
    ?atom : SIGNED_NUMBER -> number
    
    ?expr : expr SUM_OP atom -> sum_op
          | atom
        
    ?pipe : pipe "|>" expr
          | expr
    
    SUM_OP : /[+-]/
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

"""

In [78]:
class PipelineTransformer(InlineTransformer):
    number = float
        
    def pipe(self, expr_left, expr_right):  
        return ('+', expr_left, expr_right)
    
    def sum_op(self, left, op, right):
        return (str(op), left, right)
    

In [79]:
pipeline_parser = Lark(grammar, parser="lalr")
pipeline = pipeline_parser.parse
transformer = PipelineTransformer()

In [87]:
res = pipeline("1+1 |> -9 |> 1 |> -8 |> 2+2")

In [88]:
print(res.pretty())

pipe
  pipe
    pipe
      pipe
        sum_op
          number	1
          +
          number	1
        number	-9
      number	1
    number	-8
  sum_op
    number	2
    +
    number	2



In [89]:
t = transformer.transform(res)

In [90]:
def eval_expr(expr):
    if isinstance(expr, float):
        return expr
    
    head, *args = expr

    if isinstance(head, tuple):
        return eval_expr(head) + eval_expr(args)
    elif head == '+':
        x, y = args
        return eval_expr(x) + eval_expr(y)
    elif head == '-':
        x, y = args
        return eval_expr(x) - eval_expr(y)

    else:
        raise ValueError('argumento inválido para S-expression: %r' % head)

In [91]:
eval_expr(t)

-10.0